<a href="https://colab.research.google.com/github/saracmbr/LangChain/blob/main/langchain_CommaSeparatedListOutputParser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet langchain
%pip install --upgrade --quiet huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.8/120.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 5.7 MB/s eta 0:00:00


In [ ]:
# get a token: https://huggingface.co/docs/api-inference/quicktour#get-your-api-token

from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()

··········


In [ ]:
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [ ]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.output_parsers import  CommaSeparatedListOutputParser
from langchain.prompts import HumanMessagePromptTemplate , ChatPromptTemplate , SystemMessagePromptTemplate , PromptTemplate
from langchain.chains import LLMChain

In [ ]:
repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"


llm = HuggingFaceEndpoint(
    repo_id=repo_id, temperature=0.8, huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
question = "Give me 10 characterctics of chickens"
#"give me 5 ice cream flavours"

template = """Question: {question}

Format_instructions: {format}

Answer should not have any numbers in the content. Answer should not have any \n in the content.Avoid non-English words in the content.
"""

prompt = PromptTemplate.from_template(template)


In [ ]:
from langchain_core.runnables import Runnable
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
output_parser = CommaSeparatedListOutputParser()
format_instructions= output_parser.get_format_instructions()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [ ]:

#llm_chain = LLMChain(prompt=prompt, llm=llm)

#response = llm_chain.invoke({

 #          "question": question,
 #           "format": format_instructions
#           })

chain: Runnable =   prompt | llm | output_parser
response = chain.invoke({"question": question,
           "format": format_instructions})

response


['Answer: `feathers',
 'clucking',
 'lays_eggs',
 'pecks',
 'roosts',
 'waddles',
 'omnivorous',
 'brooding',
 'crowing',
 'social`']

In [ ]:
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="Give me 5 {subject}.\n{format_instructions}.\n{instruction}",
    input_variables=["subject","instruction"],
    partial_variables={"format_instructions": format_instructions}

)

In [ ]:

chain: Runnable =   prompt | llm | output_parser
output = chain.invoke({"subject" : "characteristics of chickens"})

print(type(output))
print(output)


<class 'list'>
['1. feathered\n2. bipedal\n3. omnivorous\n4. oviparous\n5. gallinaceous']


In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

response = llm_chain.invoke({
          "subject" : "winter flower name",
          "instruction": """your answer should not have any numbers in it. you should not have any \n in the content.Avoid non-English words in the content.

        """

           })

response


{'subject': 'winter flower name',
 'instruction': 'your answer should not have any numbers in it. you should not have any \n in the content.Avoid non-English words in the content.\n         \n        ',
 'text': ' Here is a list of 5 winter flowers:\n\n         Snowdrop, Pansy, Winter Jasmine, Cyclamen, Hellebore.'}

In [ ]:
print(response['text'])

 Here is a list of 5 winter flowers:

         Snowdrop, Pansy, Winter Jasmine, Cyclamen, Hellebore.


In [ ]:
output_parser.parse(response['text'])


['Here is a list of 5 winter flowers:\n\n         Snowdrop',
 'Pansy',
 'Winter Jasmine',
 'Cyclamen',
 'Hellebore.']